# **What are we going to do:**

1. Craw data poemtry
2. Preprocessing data, and write **DataLoader** ( return tokenizer of poemtry)
3. Initialize Trainer with TrainingArguments and GPT-2 model
4. Train and save the model
5. test the model

 *CHECK CUDA, INSTALL LIBRARY NECCESSARY*

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
path = '/content/drive/MyDrive/poem'
os.chdir(path)
!ls

cached_lm_PhobertTokenizer_254_test.txt        deobietdelamgi  sample.txt      train.txt
cached_lm_PhobertTokenizer_254_test.txt.lock   gpt2-poem       save_modelGPT2
cached_lm_PhobertTokenizer_254_train.txt       poem.ipynb      test.txt
cached_lm_PhobertTokenizer_254_train.txt.lock  raw_data        train3.txt


In [3]:
# !pip install torch==1.8.1 torchvision==0.9.1 torchtext==0.9.1

In [4]:
!pip3 install fairseq
!pip3 install fastbpe
!pip3 install vncorenlp
!pip install -U transformers
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 22.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.5/286.5 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11289880 sha256=9bdc201335106b6b253929366c841957b626f4d4e2216266a75e872f6370247a
  Stored in directory: /root/.cache/pip/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b9502a0ac81b2448a1196159dacc32
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.wh

# **Step1 : Craw data in internet and save 2 file train, test**

In [5]:
import re
def standardize_data(row):
    # remove stopword
    # Remove . ? , at index final
    row = re.sub(r"[\.,\?]+$-", "", row)
    row = re.sub(" \d+", " ", row)
    # Remove all . , " ... in sentences
    row = row.replace(",", "").replace(".", "") \
        .replace(";", "").replace("“", "") \
        .replace(":", "").replace("”", "") \
        .replace('"', "").replace("'", "") \
        .replace("!", "").replace("?", "") \
        .replace("-", "").replace("?", "") \
        .replace("_", "").replace("*", "") \
        .replace("(", "").replace(")", "") \
        .replace("0", "").replace("1", "") \
        .replace("2", "").replace("3", "") \
        .replace("4", "").replace("5", "") \
        .replace("6", "").replace("7", "") \
        .replace("8", "").replace("9", "") \

    row = row.strip()
    return row

def process_data(path_input, path_out):
    with open(path_input, 'r') as f:
        lines = f.readlines() # list
        for l in lines :
            process = standardize_data(l)
            with open(path_out, 'a+') as f1 :
                  f1.write(process +'\n')
    f.close()
    f1.close()

In [6]:
ly_bach_2 = '/content/drive/MyDrive/poem/raw_data/ly_bach_2.txt'
ly_bach = '/content/drive/MyDrive/poem/raw_data/ly_bach.txt'
nguyen_trai = '/content/drive/MyDrive/poem/raw_data/nguyen_trai.txt'

In [7]:
process_data(path_input =ly_bach_2, path_out ='train.txt')
process_data(path_input =ly_bach, path_out ='test.txt')
process_data(path_input =nguyen_trai, path_out ='train3.txt')

In [8]:
train_path = '/content/drive/MyDrive/poem/train.txt'
test_path = '/content/drive/MyDrive/poem/test.txt'

# 2 direct load Pretrain Model PhoBERT :




# Load pretrainModel PhoBERT use transformer

In [9]:
import torch
from transformers import AutoModel, AutoTokenizer

phoBERT = AutoModel.from_pretrained("vinai/phobert-base")
custokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
# ------add token ('\n') to enter lines --------#
custokenizer.add_tokens('\n')
# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
line = "Debug là việc thường xuyên của delevoper."

input_ids = torch.tensor([custokenizer.encode(line)])
with torch.no_grad():
    features = phoBERT(input_ids)  # Models outputs are now tuples


## **Step2 . Code DATALOADER**


*   Create dataset read file train, test :return tokenizers ( len =256)
*   Dataloader



In [10]:
#--------------Create Dataset----------------#
import os
import torch
from torch.utils.data.dataset import Dataset
from transformers.tokenization_utils import PreTrainedTokenizer
from filelock import FileLock
from transformers.utils import logging
from typing import Dict, List, Optional
import pickle
import random
import time
logger = logging.get_logger(__name__)

class PoemDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach
    soon.
    Parameters:
    ----------
    tokenizers : is pretrain tokenizer of PhoBERT
    file_path  : path to file train, test
    block_size : size of 1 block , optinal
    cache_dir  : just load 1 once and saved

    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        cache_dir: Optional[str] = None,
    ):
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"
        block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            cache_dir if cache_dir is not None else directory,
            "cached_lm_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )

        # -----------Make sure only the first process in distributed training processes the dataset,----------------#
        # ---------------------------------------and the others will use the cache------------------------#
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):

            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )

            else:
                logger.info(f"Creating features from dataset file at {directory}")

                self.examples = []
                with open(file_path, encoding="utf-8") as f:
                    text = f.read()
                #-----convert text to tokenizers----------------------------#
                '''
                1. Convert word -> subword (tokenizer.tokenize(text))
                2. COnvert subword -> number (tokenizer.convert_tokens_to_ids)
                '''
                tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

                # ------------- Truncate in block of block_size-----------------#
                #-----------Beacuse add_token('\n') -> inds = 64001------------#
                #--------If len(block_size)>256 so cut and add_special_tokens (<s>, </s>)---------------#
                i = 0
                while i < len(tokenized_text) - block_size + 1:
                    inds = tokenized_text[i : i + block_size]
                    for j in range(0, len(inds)):
                        if inds[j]==64001:
                            inds = inds[j+1:] #remove the first \n
                            break
                    for j in range(len(inds)-1, 0, -1):
                        if inds[j]==64001:
                            inds = inds[:j-1] #remove \n
                            break
                    i += len(inds)
                    self.examples.append(
                        tokenizer.build_inputs_with_special_tokens(inds)
                    )

                # Note that we are losing the last truncated example here for the sake of simplicity (no padding)
                # If your dataset is small, first you should loook for a bigger one :-) and second you
                # can change this behavior by adding (model specific) padding.

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)

In [11]:
#-----------Load dataset-----------------------#
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling, LineByLineWithSOPTextDataset

def load_dataset(train_path, test_path, custokenizer):
    train_dataset = PoemDataset(
          tokenizer=custokenizer,
          file_path=train_path,
          block_size= 256)

    test_dataset = PoemDataset(
          tokenizer=custokenizer,
          file_path=test_path,
          block_size=256)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=custokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,custokenizer)

In [12]:
#-----------Test dataloader----------------#
print(len(test_dataset))
print(len(train_dataset))
#-------------Test decode to sentence ---------------#
# print(custokenizer.decode(test_dataset[1]))

39
57


# **Step 3. Initialize Trainer with TrainingArguments and GPT-2 model**

In [13]:
from transformers import Trainer, TrainingArguments, GPT2Config, GPT2LMHeadModel

In [14]:

#--------------------------Load  pretrain model GPT-2--------------------#
model_gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')
# vntq.save_pretrained('/content/drive/MyDrive/BERT/save_modelGPT2/')


In [15]:
model_gpt2.config

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.34.1",
  "use_cache": true,
  "vocab_size": 50257
}

In [16]:
model_gpt2

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [17]:
# Check linear layer final of gpt2
model_gpt2.lm_head

Linear(in_features=768, out_features=50257, bias=False)

In [18]:
# Check weights of linear layer final
model_gpt2.lm_head.weight

Parameter containing:
tensor([[-0.1101, -0.0393,  0.0331,  ..., -0.1364,  0.0151,  0.0453],
        [ 0.0403, -0.0486,  0.0462,  ...,  0.0861,  0.0025,  0.0432],
        [-0.1275,  0.0479,  0.1841,  ...,  0.0899, -0.1297, -0.0879],
        ...,
        [-0.0445, -0.0548,  0.0123,  ...,  0.1044,  0.0978, -0.0695],
        [ 0.1860,  0.0167,  0.0461,  ..., -0.0963,  0.0785, -0.0225],
        [ 0.0514, -0.0277,  0.0499,  ...,  0.0070,  0.1552,  0.1207]],
       requires_grad=True)

In [19]:
# Random weights => fine-turning model
rand_weight = torch.rand(model_gpt2.lm_head.weight.shape)
print(rand_weight)

tensor([[0.5578, 0.0683, 0.5321,  ..., 0.9380, 0.2255, 0.9171],
        [0.5148, 0.8834, 0.0823,  ..., 0.7304, 0.2232, 0.1614],
        [0.9170, 0.6275, 0.8061,  ..., 0.3752, 0.2179, 0.5170],
        ...,
        [0.1827, 0.0198, 0.7992,  ..., 0.6873, 0.8503, 0.4960],
        [0.2927, 0.6078, 0.5463,  ..., 0.4657, 0.0964, 0.9243],
        [0.6226, 0.4553, 0.5616,  ..., 0.3404, 0.9098, 0.5198]])


In [20]:
model_gpt2.lm_head.weight = torch.nn.parameter.Parameter(rand_weight)

In [21]:
model_gpt2.lm_head.weight

Parameter containing:
tensor([[0.5578, 0.0683, 0.5321,  ..., 0.9380, 0.2255, 0.9171],
        [0.5148, 0.8834, 0.0823,  ..., 0.7304, 0.2232, 0.1614],
        [0.9170, 0.6275, 0.8061,  ..., 0.3752, 0.2179, 0.5170],
        ...,
        [0.1827, 0.0198, 0.7992,  ..., 0.6873, 0.8503, 0.4960],
        [0.2927, 0.6078, 0.5463,  ..., 0.4657, 0.0964, 0.9243],
        [0.6226, 0.4553, 0.5616,  ..., 0.3404, 0.9098, 0.5198]],
       requires_grad=True)

In [22]:
'''
Because GPT2 has vocabulary_size 50257 and (wte): Embedding(50257, 768)
So  convert vocabulary_size= 64002, Embedding(64002, 768)
'''
task_gpt2 = {"text-generation": {"do_sample": True, "max_length": 256}} #edit output size
config_gpt2 = configuration = GPT2Config(vocab_size=64002, n_positions=258, n_ctx=258,
                           task_specific_params=task_gpt2,
                           eos_token_id = 2,
                           bos_token_id = 0,
                           pad_token_id = 1,
                           sep_token_id = 2,
                          #  eos_token_id=custokenizer.eos_token_id,
                          #  bos_token_id=custokenizer.bos_token_id,
                          #  pad_token_id=custokenizer.pad_token_id,
                          #  sep_token_id=custokenizer.sep_token_id
                           )
model_gpt2 = GPT2LMHeadModel(config_gpt2)
model_gpt2


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(64002, 768)
    (wpe): Embedding(258, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=64002, bias=False)
)

In [23]:
# Check model gp2 custormer
model_gpt2.config

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 258,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 258,
  "pad_token_id": 1,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "sep_token_id": 2,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 256
    }
  },
  "transformers_version": "4.34.1",
  "use_cache": true,
  "vocab_size": 64002
}

In [24]:
#save model_gpt2 (vocabulary_size =64002)
model_gpt2.save_pretrained('/content/drive/MyDrive/poem/save_modelGPT2/')

## NEW **MODEL**

In [25]:
task = {"text-generation": {"do_sample": True, "max_length": 256}} #edit output size

In [26]:
configuration = GPT2Config(vocab_size=64002, n_positions=258, n_ctx=258,
                           task_specific_params=task,
                           eos_token_id = 2,
                           bos_token_id = 0,
                           pad_token_id = 1,
                           sep_token_id = 2,
                          #  eos_token_id=custokenizer.eos_token_id,
                          #  bos_token_id=custokenizer.bos_token_id,
                          #  pad_token_id=custokenizer.pad_token_id,
                          #  sep_token_id=custokenizer.sep_token_id
                           )
poem = GPT2LMHeadModel(configuration)

In [27]:
poem

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(64002, 768)
    (wpe): Embedding(258, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=64002, bias=False)
)

In [28]:
# Load weights of model_gpt2 ( random weights)
load_model_gpt2 = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/poem/save_modelGPT2/')
poem.load_state_dict(load_model_gpt2.state_dict())

<All keys matched successfully>

In [29]:
#-----------Print process training ------------#
from transformers.trainer_callback import TrainerCallback
from transformers import pipeline
class PrinterCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, model=None, **kwargs):
        if int(state.epoch)%10==0:
            pipe = pipeline('text-generation', model=model, tokenizer=custokenizer, device=0)
            with open("/content/drive/MyDrive/poem/sample.txt", "w") as f:
                f.write(pipe('<s> Sàng tiền minh nguyệt quang ')[0]['generated_text'])
                f.write("\n===========================================\n")
                f.close()

In [30]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/poem/gpt2-poem", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=100, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    save_steps=1000, # after # steps model is saved
    save_total_limit = 2, # delete other checkpoints
    warmup_steps=200,    # number of warmup steps for learning rate scheduler
    logging_dir='/content/drive/MyDrive/BERT/gpt2-poem/logs', # directory for storing logs
    logging_steps=200,
    )

## **Step 4. TRAIN MODEL**

In [31]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
print(torch.__version__)
torch.cuda.is_available()

2.1.0+cu118


True

In [32]:
# device = torch.device('cpu')
trainer = Trainer(
    model=poem, # GPT2
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks = [PrinterCallback],

)

In [33]:
# -------Train and save model-----------#
trainer.train()
trainer.save_model()

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Step,Training Loss
200,5.986900
400,0.578900
600,0.064300
800,0.028200


## **Step 4. TEST MODEL**

In [36]:
#-------Load model saved-----------------#
from transformers import pipeline
poem = pipeline('text-generation', model="/content/drive/MyDrive/poem/gpt2-poem", tokenizer=custokenizer)

In [37]:
#Test
a = poem('nguyệt')

In [38]:
x = a[0]['generated_text']
print(x)

nguyệt Phi tại bạch vân đoan 
 Tiên nhân thuỳ lưỡng túc 
 Quế thụ tác đoàn đoàn 
 Bạch thố đảo dược thành 
 Vấn ngôn dữ thuỳ xan 
 Thiềm thừ thực viên ảnh 
 Đại minh dạ dĩ tàn 
 Nghệ tích lạc cửu ô 
 Thiên nhân thanh thả an 
 Âm tinh thử luân hoặc 
 Khứ khứ bất túc quan 
 Ưu lai kỳ như hà 
 Thê sảng tồi tâm can 
 
 Thiên tân tam nguyệt thì 
 Thiên môn đào dữ lý 
 Triêu vi đoạn trường hoa 
 Mộ trục đông lưu thuỷ 
 Tiền thuỷ phục hậu thuỷ 
 Cổ kim tương tục lưu 
 Tân nhân phi cựu nhân 
 Niên niên kiều thượng du 
 Kê minh hải sắc động 
 Yết đế la công hầu 
 Nguyệt lạc tây Thượng Dương 
 Dư huy bán thành lâu 
 Y quan chiếu vân nhật 
 Triều hạ tán hoàng châu 
 Yên mã như phi long 
 Hoàng kim lạc mã đầu 
 Hành nhân giai tịch dị 
 Chí khí hoành Tung khâu 
 Nhập môn thướng cao đường 
 Liệt đỉnh thác trân tu 
 Hương phong dẫn triệu vũ 
 Thanh quản tuỳ Tề âu 
 Thất thập tử uyên ương 
 Song song hí đình u 
 Hành lạc tranh trú dạ 
 Tự ngôn độ thiên thu 
 Công thành thân bất
